In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from __future__ import print_function, division

from sympy import S, Expr, Mul,Add, Pow, symbols, Function, Derivative, simplify
from sympy.physics.quantum import Operator
from sympy.printing.latex import print_latex
from sympy.printing.pretty.stringpict import prettyForm


from sympy import S, Expr, Mul, Add, Pow, Rational
from sympy.printing.pretty.stringpict import prettyForm

from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum.operator import Operator

from sympy.core.decorators import _sympifyit, call_highest_priority
from sympy.core.function import UndefinedFunction

In [3]:
class PoissonBracket(Expr):
    """
    Operator class for the Poisson Bracket - allows to have it as a symbolic operator
    """
    
#     _op_priority = 11.0
    _is_commutative = False
    
    def __new__(cls,A,B,debug=True,**kwargs):
        # before return the object, do basic
        # testing and formatting of the bracket.
        # - orders arguments and updated sign
        # - returns S.Zero if:
        #    * one of the argumnents is missing
        #    * the arguments are equal
        #    * one of the arguments is a sympy number
        r = cls.eval(A,B,debug=debug,**kwargs)
        
        # if a valid object is returned from eval
        # than return this object
        if r is not None:
            if debug:
                print("Eval returned a value")
            return r
        
        # if eval has not returned anything
        # create a new object and return that one.
        obj = Expr.__new__(cls,A,B)
        obj.A = A
        obj.B = B
        
        # if independent coordinates and momenta are
        # provided store them in the object
        obj.indep_coords = kwargs.get("coords",None)
        obj.indep_mom    = kwargs.get("mom",None)
        
        # store if the ordering has been reversed
        # this is used in the doit method, otherwise
        # the sign is wrong.
        obj.order = 1
        
        # set debug to true to get intermediate outputs
        # for code debugging
        obj.debug = debug
        
        return obj
    
    @classmethod
    def eval(cls,a,b,debug, **kwargs):
        # check if both arguments of the
        # bracket are given otherwise return Zero
        if not (a and b):
            if debug:
                print("Missing term")
            return S.Zero
        
        # check if both arguments of the bracket
        # are equal, if yes return Zero
        if a==b:
            if debug:
                print("Bracket of equal aruments is Zero.")
            return S.Zero
        
        # check if one of the bracket arguments is
        # a sympy number, if yes return Zero.
        if a.is_number or b.is_number:
            if debug:
                print("At least one argument is a number, bracket is Zero")
            return S.Zero
        
        # order the elements
        if isinstance(a,UndefinedFunction):
            tmp = symbols("tmp")
            if isinstance(b, UndefinedFunction):
                if a(tmp).compare(b(tmp)) ==1:
                    if debug:
                        print("Interchanging order of arguments and put minus sign in front of bracket.")
                    nob = cls(b,a,**kwargs)
                    nob.order = -1
                    return S.NegativeOne*nob
            else:
                if a(tmp).compare(b) == 1:
                    if debug:
                        print("Interchanging order of arguments and put minus sign in front of bracket.")
                    nob = cls(b,a, **kwargs)
                    nob.order = -1
                    return S.NegativeOne*nob
        elif isinstance(b, UndefinedFunction):
            tmp = symbols("tmp")
            if a.compare(b(tmp)) ==1:
                if debug:
                    print("Interchanging order of arguments and put minus sign in front of bracket.")
                nob = cls(b,a,**kwargs)
                nob.order = -1
                return S.NegativeOne*nob
        else:
                if a.compare(b) ==1:
                        if debug:
                            print("Interchanging order of arguments and put minus sign in front of bracket.")
                        nob = cls(b,a,**kwargs)
                        nob.order = -1
                        return S.NegativeOne*nob
            
                
    @property
    def free_symbols(self):
        return self.A.free_symbols.union(self.B.free_symbols)
    
    @property
    def expr(self):
#         print(self.indep_coords)
        if (self.indep_coords and self.indep_mom):
            hp = S(0)
            for qi, pi in zip(self.indep_coords, self.indep_mom):
                hp += self.A.diff(qi) * self.B.diff(pi)
                hp -= self.A.diff(pi) * self.B.diff(qi)
            return hp
        else:
            return self
        
    def _eval_derivative(self, symbol):
        """
        Necessary to get the full expanded expression
        when one of the arguments is itself a PoissonBracket.
        In other words to allow evalutation of nested Poisson Brackets.
        """
        new_expr = self.expr.diff(symbol)
        return new_expr
    
    def doit(self, debug=True, **hints):
        """ Evaluate commutator """
        A = self.args[0]
        B = self.args[1]
        order = self.order
        if debug:
                print(self)
                print(self.__class__)
                print(self.args)
                try:
                    print(self.expr)
                except:
                    print("expr failed")
                    pass
                
        if self.order == 1:
            if debug:
                print("order 1")
            if isinstance(A, Operator) and isinstance(B, Operator):
                try:
                    comm = A._eval_commutator(B, **hints)
                except NotImplementedError:
                    try:
                        comm = -1*B._eval_commutator(A, **hints)
                    except NotImplementedError:
                        comm = None
                if comm is not None:
                    return comm.doit(**hints)
                
            if isinstance(A, UndefinedFunction) or isinstance(B, UndefinedFunction):
                if debug:
                    print("One is undefined function")
                return self
            
            elif isinstance(A,Function) or isinstance(B,Function):
                if debug:
                    print("in Function")
                h = self.expr 
                print(self.expr)
                return h
            
            return self.expr
        
        else:
            if debug:
                print("order -1")
                
            if isinstance(A, UndefinedFunction) or isinstance(B, UndefinedFunction):
                if debug:
                    print("One is undefined function")
                    print(self)
                h = self
                return h
          
            elif isinstance(A,Function) or isinstance(B,Function):
                if debug:
                    print("in Function")
                    print(self.__class__)
                h = self.expr 
                print("expr:",self.expr)
                if isinstance(A, PoissonBracket) : print(A.indep_coords);
                if isinstance(B, PoissonBracket) : print(B.indep_coords);
               
                return h
            
            return S.NegativeOne * PoissonBracket(A,B,coords=self.indep_coords,mom=self.indep_mom).expr
    
    def _eval_expand_commutator(self, **hints):
        A = self.A #self.args[0]
        B = self.B #self.args[1]

        if isinstance(A, Add):
            # [A + B, C]  ->  [A, C] + [B, C]
            sargs = []
            for term in A.args:
                comm = PoissonBracket(term, B)
                print(comm.__class__)
                if isinstance(comm, PoissonBracket):
                    comm = comm._eval_expand_commutator()
                sargs.append(comm)
#             print(sargs)
            return Add(*sargs)
        elif isinstance(B, Add):
            # [A, B + C]  ->  [A, B] + [A, C]
            sargs = []
            for term in B.args:
                comm = PoissonBracket(A, term)
                if isinstance(comm, PoissonBracket):
                    comm = comm._eval_expand_commutator()
                sargs.append(comm)
            return Add(*sargs)
        elif isinstance(A, Mul):
            # [A*B, C] -> A*[B, C] + [A, C]*B
            a = A.args[0]
            b = Mul(*A.args[1:])
            c = B
            comm1 = PoissonBracket(b, c)
            comm2 = PoissonBracket(a, c)
            if isinstance(comm1, PoissonBracket):
                comm1 = comm1._eval_expand_commutator()
            if isinstance(comm2, PoissonBracket):
                comm2 = comm2._eval_expand_commutator()
            first = Mul(a, comm1)
            second = Mul(comm2, b)
#             print(first,second)
            return Add(first, second)
        elif isinstance(B, Mul):
            # [A, B*C] -> [A, B]*C + B*[A, C]
            a = A
            b = B.args[0]
            c = Mul(*B.args[1:])
            comm1 = PoissonBracket(a, b)
            comm2 = PoissonBracket(a, c)
            if isinstance(comm1, PoissonBracket):
                comm1 = comm1._eval_expand_commutator()
            if isinstance(comm2, PoissonBracket):
                comm2 = comm2._eval_expand_commutator()
            first = Mul(comm1, c)
            second = Mul(b, comm2)
            return Add(first, second)
        elif isinstance(A, Pow):
            # [A**n, C] -> A**(n - 1)*[A, C] + A**(n - 2)*[A, C]*A + ... + [A, C]*A**(n-1)
            return self._expand_pow(A, B, 1)
        elif isinstance(B, Pow):
            # [A, C**n] -> C**(n - 1)*[C, A] + C**(n - 2)*[C, A]*C + ... + [C, A]*C**(n-1)
            return self._expand_pow(B, A, -1)
        return self
    
    def _expand_pow(self, A, B, sign):
        exp = A.exp
        if not exp.is_integer or not exp.is_constant() or abs(exp) <= 1:
            # nothing to do
            return self
        base = A.base
        if exp.is_negative:
            base = A.base**-1
            exp = -exp
        comm = PoissonBracket(base, B).expand(commutator=True)

        result = base**(exp - 1) * comm
        for i in range(1, exp):
            result += base**(exp - 1 - i) * comm * base**i
        return sign*result.expand()
    
    
    
    def _latex(self, printer, *args):
#         print(tuple([
#             printer._print(arg.func, *args) if arg.__class__.__name__ != "PoissonBracket" else  printer._print(arg, *args) for arg in self.args[:2] ]))
        news = []
        for arg in self.args[:2]:
#             print(type(arg))
            if isinstance(arg,Add):
                news.append(" + ".join([printer.doprint(a.func, *args) if a.is_Function  else  printer.doprint(a, *args) for a in arg.args]))
            
            elif isinstance(arg,PoissonBracket):
                news.append(printer.doprint(arg,*args))
#                 news.append("\\lbrace %s,%s\\rbrace" %tuple([
#             printer.doprint(a.func, *args) if a.is_Function and not isinstance(a,UndefinedFunction)  else  printer.doprint(a, *args) for a in arg.args[:2] ]))
            
            elif isinstance(arg,Mul):
                 news.append(" * ".join([printer.doprint(a.func, *args) if a.is_Function  else  printer.doprint(a, *args) for a in arg.args]))
#                 print([a for a in arg.args])
#                 news.append(" * ".join([ 
#                     printer._print(a.func, *args) 
#                     if a.is_Function  and not (isinstance(a,UndefinedFunction))
#                     else  printer._print(a, *args) for a in args
#                 ]))
            elif isinstance(arg,Pow):
                if isinstance(arg.args[0], UndefinedFunction):
                    news.append("{}^{}".format([printer.doprint(a) for a in arg.args]))
                elif isinstance(arg.args[0], Function):
                    news.append("{}^{}".format(printer.doprint(arg.args[0].func,*args), printer.doprint(arg.args[1],*args)))
                else:
                    news.append(printer.doprint(arg, *args))
                
            elif isinstance(arg, Function) and not (isinstance(arg,UndefinedFunction)):
                news.append(printer.doprint(arg.func, *args))
                                                    
            else: # isinstance(arg, Function) and isinstance(arg,UndefinedFunction):
                news.append(printer.doprint(arg, *args))
    
        print(news)
        return "\\lbrace %s,%s\\rbrace" %tuple(news)
        
        return "\\lbrace %s,%s\\rbrace" %tuple([
            printer._print(arg.func, *args) if arg.is_Function  and not (isinstance(arg,UndefinedFunction))
            else  printer._print(arg, *args) 
            for arg in self.args[:2] ])
#         return "\\lbrace %s,%s\\rbrace" % tuple([
#             printer._print(arg, *args) for arg in self.args[:2]])

# Tests Setup

In [4]:
A = Function("A", commutative=False)
B = Function("B", commutative=False)
C = Function("C", commutative=False)
D = Function("D", commutative=False)

x, px, y, py = symbols("x px y py")

half = Rational(1, 2)

# Tests

In [5]:
_file__ = "Untitled.ipynb"
import ipytest

import pytest
ipytest.autoconfig()

## Single Brackets

In [6]:
%%run_pytest[clean]

import pytest

single_pb_fail = [
    ((1,2)   , 0),
    ((1,half), 0),
]

single_pb_pass = [
    ((S.One   , half),{}, 0),
    ((A       , half),{}, 0),
    ((A(x)    , half),{}, 0),
    ((A(x,px) , half),{}, 0),
    ((S.One   , half),{"coords":[x]}, 0),
    ((A       , half),{"coords":[x]}, 0),
    ((A(x)    , half),{"coords":[x]}, 0),
    ((A(x,px) , half),{"coords":[x]}, 0),
    (reversed((S.One   , half)),{}, 0),
    (reversed((A       , half)),{}, 0),
    (reversed((A(x)    , half)),{}, 0),
    (reversed((A(x,px) , half)),{}, 0),
    (reversed((S.One   , half)),{"coords":[x]}, 0),
    (reversed((A       , half)),{"coords":[x]}, 0),
    (reversed((A(x)    , half)),{"coords":[x]}, 0),
    (reversed((A(x,px) , half)),{"coords":[x]}, 0),
]

single_pb_doit_fail = [
    
]

single_pb_doit_pass = [
    # Brackets doit returns self - no indep coords/mom to take der.
    ((A       , B       ),{}, PoissonBracket(A, B)),
    ((A(x,px) , B       ),{}, PoissonBracket(A(x,px), B)),
    ((A       , B(x,px) ),{}, PoissonBracket(A, B(x,px))),
    ((A(x,px) , B(x,px) ),{}, PoissonBracket(A(x,px), B(x,px))),
    ((x **2   , x + px  ),{}, PoissonBracket(x ** 2, x + px)),
    
    (reversed((A       , B       )),{}, PoissonBracket(B, A)),
    (reversed((A(x,px) , B       )),{}, PoissonBracket(B, A(x,px))),
    (reversed((A       , B(x,px) )),{}, PoissonBracket(B(x,px),A)),
    (reversed((A(x,px) , B(x,px) )),{}, PoissonBracket(B(x,px),A(x,px))),
    
    # Brackets actually performing the doit
    ((A       , B       ),{"coords":[x]}, PoissonBracket(A, B)),
    ((A(x,px) , B       ),{"coords":[x]}, PoissonBracket(A(x,px), B)),
    ((A       , B(x,px) ),{"coords":[x]}, PoissonBracket(A, B(x,px))),
    ((A(x,px) , B(x,px) ),{"coords":[x]}, PoissonBracket(A(x,px), B(x,px))),
    
    ((x **2     , x + px     ), {"coords":[x]  , "mom": [px]   }, 2*x),
    ((A(x, px)  , B(x, y, py)), {"coords":[x]  , "mom": [px]   }, -Derivative(A(x, px), px)*Derivative(B(x, y, py), x)),
    ((A(x,y, px), B(x, y, py)), {"coords":[x]  , "mom": [px]   }, -Derivative(A(x, y, px), px)*Derivative(B(x, y, py), x)),
    ((A(x, px)  , B(x, y, py)), {"coords":[x,y], "mom": [px]   }, -Derivative(A(x, px), px)*Derivative(B(x, y, py), x)),
    ((A(x, px)  , B(x, y, py)), {"coords":[x,y], "mom": [px,py]}, -Derivative(A(x, px), px)*Derivative(B(x, y, py), x)),
    ((A(x,y, px), B(x, y, py)), {"coords":[x,y], "mom": [px,py]}, -Derivative(A(x, y, px), px)*Derivative(B(x, y, py), x) +
     Derivative(A(x, y, px), y)*Derivative(B(x, y, py), py))
]
@pytest.mark.parametrize('input,expected',single_pb_fail)
def test_single_pb_parametrized_fail(input, expected):
    with pytest.raises(AttributeError):
        test = PoissonBracket(*input)
        

@pytest.mark.parametrize('input,kwargs,expected',single_pb_pass)
def test_single_pb_parametrized_pass(input, kwargs, expected):
    test = PoissonBracket(*input,**kwargs)
    assert test == expected
    
    
@pytest.mark.parametrize('input,kwargs,expected',single_pb_doit_pass)
def test_single_pb_doit_parametrized_pass(input, kwargs, expected):
    test = PoissonBracket(*input,**kwargs)
    assert test.doit() == expected

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
...........

/home/mti/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3144: UserWarning: AST transformer <ipytest._pytest_support.RewriteAssertTransformer object at 0x7f822c666cc0> threw an error. It will be unregistered.
  warn("AST transformer %r threw an error. It will be unregistered." % transformer)


..........................  [100%]
37 passed in 0.16s


## Double Brackets

In [7]:
%%run_pytest[clean]

import pytest

double_pb_fail = [
    
]

double_pb_pass = [
    ((S.One   , half), (S.One,) ,{}            , {}, 0),
    ((A       , half), (S.One,) ,{}            , {}, 0),
    ((A(x)    , half), (S.One,) ,{}            , {}, 0),
    ((A(x,px) , half), (S.One,) ,{}            , {}, 0),
    ((S.One   , half), (S.One,) ,{"coords":[x]}, {}, 0),
    ((A       , half), (S.One,) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (S.One,) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (S.One,) ,{"coords":[x]}, {}, 0),
    
    ((S.One   , half), (A,) ,{}            , {}, 0),
    ((A       , half), (A,) ,{}            , {}, 0),
    ((A(x)    , half), (A,) ,{}            , {}, 0),
    ((A(x,px) , half), (A,) ,{}            , {}, 0),
    ((S.One   , half), (A,) ,{"coords":[x]}, {}, 0),
    ((A       , half), (A,) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (A,) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (A,) ,{"coords":[x]}, {}, 0),
    
    ((S.One   , half), (A(x),) ,{}            , {}, 0),
    ((A       , half), (A(x),) ,{}            , {}, 0),
    ((A(x)    , half), (A(x),) ,{}            , {}, 0),
    ((A(x,px) , half), (A(x),) ,{}            , {}, 0),
    ((S.One   , half), (A(x),) ,{"coords":[x]}, {}, 0),
    ((A       , half), (A(x),) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (A(x),) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (A(x),) ,{"coords":[x]}, {}, 0),
    
    ((S.One   , half), (A(x,px),) ,{}            , {}, 0),
    ((A       , half), (A(x,px),) ,{}            , {}, 0),
    ((A(x)    , half), (A(x,px),) ,{}            , {}, 0),
    ((A(x,px) , half), (A(x,px),) ,{}            , {}, 0),
    ((S.One   , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    ((A       , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (S.One,) ,{}            , {}, 0),
    (reversed((A       , half)), (S.One,) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (S.One,) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (S.One,) ,{}            , {}, 0),
    (reversed((S.One   , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (A,) ,{}            , {}, 0),
    (reversed((A       , half)), (A,) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (A,) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (A,) ,{}            , {}, 0),
    (reversed((S.One   , half)), (A,) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (A,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (A,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (A,) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (A(x),) ,{}            , {}, 0),
    (reversed((A       , half)), (A(x),) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (A(x),) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (A(x),) ,{}            , {}, 0),
    (reversed((S.One   , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((A       , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((S.One   , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    
    ((A     , B    ), (S.One,) ,{}            , {}, 0),
]

@pytest.mark.parametrize('input1,input2,kwargs1,kwargs2,expected',double_pb_pass)
def test_double_pb_parametrized_pass(input1, input2, kwargs1, kwargs2, expected):
    pb1 = PoissonBracket(*input1, **kwargs1)
    pb2 = PoissonBracket(pb1, *input2, **kwargs2)
    assert pb2 == expected

...................................... [ 58%]
...........................            [100%]
65 passed in 0.19s


In [49]:
%%run_pytest[clean]

import pytest

double_pb_fail = [
    
]

double_pb_doit_pass = [
    ((A      , B      ), (S.One,) ,{}            , {}, 0),
    ((A      , B      ), (C    ,) ,{}            , {}, PoissonBracket(PoissonBracket(A,B),C)),
    ((A(x,px), B(x,px)), (C    ,) ,{}            , {}, PoissonBracket(PoissonBracket(A(x,px),B(x,px)),C)),
    ((x **2  , x + px ), (C    ,) ,{}            , {}, PoissonBracket(PoissonBracket(x**2,x+px),C)),
    
    ((A,B),(C,), {}, {}, S.NegativeOne * PoissonBracket(C, PoissonBracket(A, B))),
    ((A(x),B),(C,), {}, {}, S.NegativeOne * PoissonBracket(C, PoissonBracket(A(x), B))),
    ((A,B(x)),(C,), {}, {}, S.NegativeOne * PoissonBracket(C, PoissonBracket(A, B(x)))),
    ((A,B),(C(x),), {}, {}, S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A, B))),
    
    ((A,B),(C,), {"coords":[x]}, {}, S.NegativeOne * PoissonBracket(C, PoissonBracket(A, B))),
    ((A(x),B),(C,), {"coords":[x]}, {}, S.NegativeOne * PoissonBracket(C, PoissonBracket(A(x), B))),
    ((A,B(x)),(C,), {"coords":[x]}, {}, S.NegativeOne * PoissonBracket(C, PoissonBracket(A, B(x)))),
    ((A,B),(C(x),), {"coords":[x]}, {}, S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A, B))),
    
    ((A,B),(C,),{}, {"coords":[x]}, S.NegativeOne * PoissonBracket(C, PoissonBracket(A, B))),
    ((A(x),B),(C,),{}, {"coords":[x]},  S.NegativeOne * PoissonBracket(C, PoissonBracket(A(x), B))),
    ((A,B(x)),(C,),{}, {"coords":[x]},  S.NegativeOne * PoissonBracket(C, PoissonBracket(A, B(x)))),
    ((A,B),(C(x),),{}, {"coords":[x]},  S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A, B))),
    
    ((A(x),B(x)),(C,), {"coords":[x]}, {}, S.NegativeOne * PoissonBracket(C, PoissonBracket(A(x), B(x)))),
    ((A(x),B(x)),(C(x),), {"coords":[x]}, {}, 0),
    ((A(x),B(x)),(C(x),), {},{"coords":[x]}, 0),
    
    ((A(x,px),B),(C,), {}, {}, S.NegativeOne * PoissonBracket(C, PoissonBracket(A(x,px), B))),
#     ((x **2  , x + px ), (C       ,) ,{"coords":[x], "mom": [px]}, {}, PoissonBracket(PoissonBracket(x**2,x+px,coords=[x],mom=[px]),C)),
#     ((A(x,px), B(x,px)), (C(x,px) ,) ,{"coords":[x], "mom": [px]}, {}, PoissonBracket(PoissonBracket(A(x,px),B(x,px)),C(x,px))),
#     ((A(x,px), B(x,px)), (C       ,) ,{"coords":[x], "mom": [px]}, {}, PoissonBracket(PoissonBracket(A(x,px),B(x,px)),C)),
    
#     ((A(x,px), B(x,px)), (C(x,px) ,) ,{"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, S.NegativeOne*(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x))),
]

@pytest.mark.parametrize('input1,input2,kwargs1,kwargs2,expected',double_pb_doit_pass)
def test_double_pb_doit_parametrized_pass(input1, input2, kwargs1, kwargs2, expected):
    pb1 = PoissonBracket(*input1, **kwargs1)
    pb2 = PoissonBracket(pb1, *input2, **kwargs2)
    print(pb2.doit()-expected)
    assert pb2.doit() == expected
    print(pb2.doit()-expected)

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
....................                   [100%]
20 passed in 0.12s


In [48]:
test5 = PoissonBracket(A(x), B(x))
test12 = PoissonBracket(test5, C(x),coords=[x])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x), PoissonBracket(A(x), B(x)))
<class '__main__.PoissonBracket'>
(C(x), PoissonBracket(A(x), B(x)))
0
order -1
in Function
<class '__main__.PoissonBracket'>
expr: 0
[x]


0

In [8]:
test5 = PoissonBracket(A(x, px), B(x,px),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(x, px), coords=[x], mom=[px])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, px)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x, px)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))
order -1
in Function
<class '__main__.PoissonBracket'>
expr: -Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A

-(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px)))

In [22]:
test5 = PoissonBracket(A(x,px), B(x,y,px),coords=[x,y],mom=[px,py])
test12 = PoissonBracket(test5, C(y,py), coords=[x,y], mom=[px,py])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(y, py), PoissonBracket(A(x, px), B(x, y, px)))
<class '__main__.PoissonBracket'>
(C(y, py), PoissonBracket(A(x, px), B(x, y, px)))
-Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))
order -1
in Function
<class '__main__.PoissonBracket'>
expr: -Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))
[x]


Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))

In [21]:
test5 = PoissonBracket(A(x,px), B(x,y,px),coords=[x,y],mom=[px,py])
test12 = PoissonBracket(test5, C(y,py), coords=[x,y], mom=[py])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(y, py), PoissonBracket(A(x, px), B(x, y, px)))
<class '__main__.PoissonBracket'>
(C(y, py), PoissonBracket(A(x, px), B(x, y, px)))
-Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))
order -1
in Function
<class '__main__.PoissonBracket'>
expr: -Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))
[x]


Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))

In [20]:
test5 = PoissonBracket(A(x,px), B(x,y,px),coords=[x,y],mom=[px,py])
test12 = PoissonBracket(test5, C(y,py), coords=[y], mom=[py])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(y, py), PoissonBracket(A(x, px), B(x, y, px)))
<class '__main__.PoissonBracket'>
(C(y, py), PoissonBracket(A(x, px), B(x, y, px)))
-Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))
order -1
in Function
<class '__main__.PoissonBracket'>
expr: -Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))
[x]


Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))

In [18]:
test5 = PoissonBracket(A(x,px), B(x,y,px),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(y,py), coords=[y], mom=[py])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(y, py), PoissonBracket(A(x, px), B(x, y, px)))
<class '__main__.PoissonBracket'>
(C(y, py), PoissonBracket(A(x, px), B(x, y, px)))
-Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))
order -1
in Function
<class '__main__.PoissonBracket'>
expr: -Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))
[x]


Derivative(C(y, py), py)*(-Derivative(A(x, px), px)*Derivative(B(x, y, px), x, y) + Derivative(A(x, px), x)*Derivative(B(x, y, px), px, y))

In [17]:
test5 = PoissonBracket(A(x,px), B(x,px),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(y,py), coords=[y], mom=[py])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(y, py), PoissonBracket(A(x, px), B(x, px)))
<class '__main__.PoissonBracket'>
(C(y, py), PoissonBracket(A(x, px), B(x, px)))
0
order -1
in Function
<class '__main__.PoissonBracket'>
expr: 0
[x]


0

In [16]:
test5 = PoissonBracket(A(x,px), B(x,px),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(y), coords=[y], mom=[py])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(y), PoissonBracket(A(x, px), B(x, px)))
<class '__main__.PoissonBracket'>
(C(y), PoissonBracket(A(x, px), B(x, px)))
0
order -1
in Function
<class '__main__.PoissonBracket'>
expr: 0
[x]


0

In [15]:
test5 = PoissonBracket(A(x,px), B(x,px),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(x,px), coords=[x], mom=[px])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, px)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x, px)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))
order -1
in Function
<class '__main__.PoissonBracket'>
expr: -Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A

-(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px)))

In [14]:
test5 = PoissonBracket(A(x,px), B(x,px),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(x), coords=[x], mom=[px])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x), PoissonBracket(A(x, px), B(x, px)))
<class '__main__.PoissonBracket'>
(C(x), PoissonBracket(A(x, px), B(x, px)))
Derivative(C(x), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))
order -1
in Function
<class '__main__.PoissonBracket'>
expr: Derivative(C(x), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))
[x]


-Derivative(C(x), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))

In [13]:
test5 = PoissonBracket(A(x,px), B(x),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C, coords=[x], mom=[px])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C, PoissonBracket(A(x, px), B(x)))
<class '__main__.PoissonBracket'>
(C, PoissonBracket(A(x, px), B(x)))
Derivative(A(x, px), px)*Derivative(B(x), (x, 2)) - Derivative(A(x, px), (px, 2))*Derivative(B(x), x) + Derivative(A(x, px), px, x)*Derivative(B(x), x)
order -1
One is undefined function
PoissonBracket(C, PoissonBracket(A(x, px), B(x)))
['A', 'B']
['C', '\\lbrace A,B\\rbrace']


-PoissonBracket(C, PoissonBracket(A(x, px), B(x)))

In [12]:
test5 = PoissonBracket(A(x,px), B(x,px),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(x), coords=[x], mom=[px])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x), PoissonBracket(A(x, px), B(x, px)))
<class '__main__.PoissonBracket'>
(C(x), PoissonBracket(A(x, px), B(x, px)))
Derivative(C(x), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))
order -1
in Function
<class '__main__.PoissonBracket'>
expr: Derivative(C(x), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))
[x]


-Derivative(C(x), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))

In [11]:
test5 = PoissonBracket(A(x), B(x,px),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(x), coords=[x], mom=[px])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))
<class '__main__.PoissonBracket'>
(C(x), PoissonBracket(A(x), B(x, px)))
Derivative(C(x), x)*Derivative(A(x), x)*Derivative(B(x, px), (px, 2))
order -1
in Function
<class '__main__.PoissonBracket'>
expr: Derivative(C(x), x)*Derivative(A(x), x)*Derivative(B(x, px), (px, 2))
[x]


-Derivative(C(x), x)*Derivative(A(x), x)*Derivative(B(x, px), (px, 2))

In [10]:
test5 = PoissonBracket(A(x,px), B(x),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(x), coords=[x], mom=[px])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))
<class '__main__.PoissonBracket'>
(C(x), PoissonBracket(A(x, px), B(x)))
-Derivative(C(x), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x)
order -1
in Function
<class '__main__.PoissonBracket'>
expr: -Derivative(C(x), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x)
[x]


Derivative(C(x), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x)

In [9]:
test5 = PoissonBracket(A(x), B(x),coords=[x],mom=[px])
test12 = PoissonBracket(test5, C(x), coords=[x], mom=[px])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x), PoissonBracket(A(x), B(x)))
<class '__main__.PoissonBracket'>
(C(x), PoissonBracket(A(x), B(x)))
0
order -1
in Function
<class '__main__.PoissonBracket'>
expr: 0
[x]


0

In [36]:
test12.args[1].indep_coords

In [37]:
PoissonBracket(A(x, px), B(x, px)).expr

['A', 'B']


PoissonBracket(A(x, px), B(x, px))

In [38]:
test5 = PoissonBracket(A(x, px), B(x, y, py), coords=[x, y], mom=[px, py])
test12 = PoissonBracket(test5, C(x, px), coords=[x, y], mom=[px, py])
# test12.doit()==S.NegativeOne*(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x))
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
order -1
in Function
<class '__main__.PoissonBracket'>
expr: -Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
[x, y]


-(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x))

In [71]:
test12.free_symbols

{px, py, x, y}

In [201]:
test12.doit()==S.NegativeOne*(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x))


PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
order -1
in Function
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)


True

In [202]:
test12.doit(),

PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
order -1
in Function
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)


(-(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)),)

In [40]:



def test_double_pb_Function_with_var_indep_Function_with_var_indep():
    test5 = PoissonBracket(A(x, px), B(x, y, py), coords=[x, y], mom=[px, py])
    test12 = PoissonBracket(test5, C(x, px), coords=[x, y], mom=[px, py])
    res12 = (-Derivative(C(x, px), px)*
             (-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2))
              - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) 
             - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
            )
    assert test12.free_symbols == {x, px, py, y}
    assert test12.doit() == res12


def test_single_pb_sum_Function_with_var_Function_with_var_indep():
    test13 = PoissonBracket(A(x, px) + B(x, y, py), C(x, px), coords=[x, y], mom=[px, py])
    res13 = (Derivative(A(x, px), x) + Derivative(B(x, y, py), x)) * Derivative(
        C(x, px), px
    ) - Derivative(A(x, px), px) * Derivative(C(x, px), x)
    assert test13.free_symbols == {x, px, py, y}
    assert test13.doit() == res13


def test_single_pb_Function_with_var_sum_Function_with_var_indep():
    test14 = PoissonBracket(A(x, px), B(x, y, py) + C(x, px), coords=[x, y], mom=[px, py])
    res14 = -Derivative(A(x, px), px) * (
        Derivative(B(x, y, py), x) + Derivative(C(x, px), x)
    ) + Derivative(A(x, px), x) * Derivative(C(x, px), px)
    assert test14.free_symbols == {x, px, py, y}
    assert test14.doit() == res14


def test_single_pb_product_Function_with_var_Function_with_var_indep():
    test15 = PoissonBracket(A(x, px) * B(x, y, py), C(x, px), coords=[x, y], mom=[px, py])
    res15 = (
        A(x, px) * Derivative(B(x, y, py), x) + Derivative(A(x, px), x) * B(x, y, py)
    ) * Derivative(C(x, px), px) - Derivative(A(x, px), px) * B(x, y, py) * Derivative(C(x, px), x)
    assert test15.free_symbols == {x, px, py, y}
    assert test15.doit() == res15


def test_single_pb_Function_with_var_product_Function_with_var_indep():
    test16 = PoissonBracket(A(x, px), B(x, y, py) * C(x, px), coords=[x, y], mom=[px, py])
    res16 = -Derivative(A(x, px), px) * (
        B(x, y, py) * Derivative(C(x, px), x) + Derivative(B(x, y, py), x) * C(x, px)
    ) + Derivative(A(x, px), x) * B(x, y, py) * Derivative(C(x, px), px)
    assert test16.free_symbols == {x, px, py, y}
    assert test16.doit() == res16


def test_single_pb_Pow_Function_with_var_Function_with_var_indep():
    test17 = PoissonBracket(A(x, px) ** 3, B(x, y, py), coords=[x, y], mom=[px, py])
    res17 = (
        -3
        * A(x, px) ** 3
        * Derivative(A(x, px), px)
        * A(x, px) ** (-1)
        * Derivative(B(x, y, py), x)
    )
    assert test17.free_symbols == {x, px, py, y}
    assert test17.doit() == res17


def test_triple_pb_3Function_with_var_indep():
    test5 = PoissonBracket(A(x, px), B(x, y, py), coords=[x, y], mom=[px, py])
    test12 = PoissonBracket(test5, C(x, px), coords=[x, y], mom=[px, py])
    test18 = PoissonBracket(test12, D(x, y), coords=[x, y], mom=[px, py])
    res18 = -(
        (
            -Derivative(A(x, px), px) * Derivative(B(x, y, py), py, (x, 2))
            - Derivative(A(x, px), px, x) * Derivative(B(x, y, py), py, x)
        )
        * Derivative(C(x, px), px)
        + Derivative(A(x, px), (px, 2)) * Derivative(B(x, y, py), py, x) * Derivative(C(x, px), x)
    ) * Derivative(D(x, y), y) - (
        (
            -Derivative(A(x, px), px) * Derivative(B(x, y, py), (x, 2))
            - Derivative(A(x, px), px, x) * Derivative(B(x, y, py), x)
        )
        * Derivative(C(x, px), (px, 2))
        + (
            -Derivative(A(x, px), (px, 2)) * Derivative(B(x, y, py), (x, 2))
            - Derivative(A(x, px), (px, 2), x) * Derivative(B(x, y, py), x)
        )
        * Derivative(C(x, px), px)
        + Derivative(A(x, px), (px, 2)) * Derivative(B(x, y, py), x) * Derivative(C(x, px), px, x)
        + Derivative(A(x, px), (px, 3)) * Derivative(B(x, y, py), x) * Derivative(C(x, px), x)
    ) * Derivative(
        D(x, y), x
    )
    assert test18.free_symbols == {x, px, py, y}


def test_doube_pb_rsum_deep():
    test13 = PoissonBracket(A(x, px) + B(x, y, py), C(x, px), coords=[x, y], mom=[px, py])
    test19 = PoissonBracket(test13, D(x, y), coords=[x, y], mom=[px, py])
    res19 = -(
        (Derivative(A(x, px), x) + Derivative(B(x, y, py), x)) * Derivative(C(x, px), (px, 2))
        - Derivative(A(x, px), px) * Derivative(C(x, px), px, x)
        - Derivative(A(x, px), (px, 2)) * Derivative(C(x, px), x)
        + Derivative(A(x, px), px, x) * Derivative(C(x, px), px)
    ) * Derivative(D(x, y), x) - Derivative(B(x, y, py), py, x) * Derivative(
        C(x, px), px
    ) * Derivative(
        D(x, y), y
    )
    assert test19.free_symbols == {x, px, py, y}

...........FF.F.F...                   [100%]
================== FAILURES ==================
_ test_double_pb_Function_with_var_indep_Function_with_var_no_indep _

    def test_double_pb_Function_with_var_indep_Function_with_var_no_indep():
        test5 = PoissonBracket(A(x, px), B(x, y, py), coords=[x, y], mom=[px, py])
        test11 = PoissonBracket(test5, C(x, px))
        assert test11.free_symbols == {x, px, py, y}
>       assert test11.doit() == test11
E       AssertionError

<ipython-input-40-3b334680848e>:73: AssertionError
------------ Captured stdout call ------------
-1
PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
order 1
_ test_double_pb_Function_wi

In [52]:
test5 = PoissonBracket(A(x, px), B(x,px), coords=[x, y], mom=[px, py])
test12 = PoissonBracket(test5, C(x, px), coords=[x, y], mom=[px, py])
test12.doit()

TypeError: 'C' object is not callable

In [51]:
test5 = PoissonBracket(A(x, px), B(x,px), coords=[x, y], mom=[px, py])
test12 = PoissonBracket(test5,C(x,px),coords=[x],mom=[px])

TypeError: 'C' object is not callable

In [50]:
def test():
    self = test12.args[1]
    A = self.args[0]
    B = self.args[1]
    order = self.order
    debug=True
    if debug:
        print(test12)
        print(test12.__class__)
        print(test12.args)
        print(order)
        try:
            print(test12.args[1].expr)
        except:
            print("expr failed")
            pass

    if self.order == 1:
        if debug:
            print("order 1")
        if isinstance(A, Operator) and isinstance(B, Operator):
            try:
                comm = A._eval_commutator(B, **hints)
            except NotImplementedError:
                try:
                    comm = -1*B._eval_commutator(A, **hints)
                except NotImplementedError:
                    comm = None
            if comm is not None:
                return comm.doit(**hints)

        if isinstance(A, UndefinedFunction) or isinstance(B, UndefinedFunction):
            if debug:
                print("One is undefined function")
            return self

        elif isinstance(A,Function) or isinstance(B,Function):
            if debug:
                print("in Function")
            h = self.expr 
            print(self.expr)
            return h

        return self.expr
        
    else:
        if debug:
            print("order -1")

        if isinstance(A, UndefinedFunction) or isinstance(B, UndefinedFunction):
            if debug:
                print("One is undefined function")
                print(self)
            h = self
            return h

        elif isinstance(A,Function) or isinstance(B,Function):
            if debug:
                print("in Function")
                print(self.__class__)
            h = self.expr 
            print("expr:",self.expr)
            if isinstance(A, PoissonBracket) : print(A.indep_coords);
            if isinstance(B, PoissonBracket) : print(B.indep_coords);

            return h

        return S.NegativeOne * PoissonBracket(A,B,coords=self.indep_coords,mom=self.indep_mom).expr

test()

-PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
<class 'sympy.core.mul.Mul'>
(-1, PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py))))
-1
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
order -1
in Function
<class '__main__.PoissonBracket'>
expr: -Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
[x, y]


-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)

In [88]:
test5.free_symbols

{px, x}

In [89]:
test12.args[1].free_symbols

{px, x}

In [25]:
test12 = PoissonBracket(test5,C(x,px),coords=[x],mom=[px])
test12.doit()

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x)
order -1
in Function
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x)
[x, y]


-(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x))

In [21]:
test12.doit()

PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
order -1
in Function
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
[x, y]


-(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x))

In [38]:
res12 = (-Derivative(C(x, px), px)*
             (-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2))
              - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) 
             - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
            )

In [39]:
simplify(test12.doit() -res12)

PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x, y, py)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)
order 1


0

In [9]:
test12+ PoissonBracket(C(x,px),test5,coords=[x,y],mom=[px,py])

0

In [464]:
1

x, px, y, py, z, delta, eps, h , k0, k1, k2, sigma, betag ,f = symbols('x p_x y p_y z delta epsilon h k_0 k_1 k_2 sigma beta_gamma f', real=True)

2

L, beta0, gamma0 = symbols(" L beta_0 gamma_0", real=True, positive=True)

In [465]:
A = Function("A", commutative=False)
B = Function("B", commutative=False)
C = Function("C", commutative=False)
D = Function("D", commutative=False)

In [466]:
test1 =PoissonBracket(A,B)
test1

['A', 'B']


PoissonBracket(A, B)

In [467]:
test1 == PoissonBracket(A,B)

True

In [468]:
test1.doit()

PoissonBracket(A, B)
<class '__main__.PoissonBracket'>
(A, B)
PoissonBracket(A, B)
order 1
One is undefined function
['A', 'B']


PoissonBracket(A, B)

In [469]:
test1.A= A(x)

In [470]:
test1.free_symbols

{x}

In [471]:
test2 = PoissonBracket(x**2,x+px)
test2.doit()

PoissonBracket(x**2, p_x + x)
<class '__main__.PoissonBracket'>
(x**2, p_x + x)
PoissonBracket(x**2, p_x + x)
order 1
['x^{2}', 'p_{x} + x']


PoissonBracket(x**2, p_x + x)

In [472]:
[i for i in test2.free_symbols][0],0

(p_x, 0)

In [473]:
test3 = PoissonBracket(A(x,px),B(y,py))
test3

['A', 'B']


PoissonBracket(A(x, p_x), B(y, p_y))

In [474]:
test3.doit()

PoissonBracket(A(x, p_x), B(y, p_y))
<class '__main__.PoissonBracket'>
(A(x, p_x), B(y, p_y))
PoissonBracket(A(x, p_x), B(y, p_y))
order 1
['A', 'B']


PoissonBracket(A(x, p_x), B(y, p_y))

In [475]:
test4 = PoissonBracket(x**2,x+px,coords=[x],mom=[px])
test4

['x^{2}', 'p_{x} + x']


PoissonBracket(x**2, p_x + x)

In [476]:
test4.doit()

PoissonBracket(x**2, p_x + x)
<class '__main__.PoissonBracket'>
(x**2, p_x + x)
2*x
order 1


2*x

In [477]:
test5 = PoissonBracket(A(x,px),B(x,y,py),coords=[x,y],mom=[px,py])
test5

['A', 'B']


PoissonBracket(A(x, p_x), B(x, y, p_y))

In [478]:
test5.expr

-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x)

In [479]:
test5.doit()

PoissonBracket(A(x, p_x), B(x, y, p_y))
<class '__main__.PoissonBracket'>
(A(x, p_x), B(x, y, p_y))
-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x)
order 1


-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x)

In [480]:
test6 = PoissonBracket(test1,C)
test6

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A, B), C)

In [481]:
test6.doit()

PoissonBracket(PoissonBracket(A, B), C)
<class '__main__.PoissonBracket'>
(PoissonBracket(A, B), C)
PoissonBracket(PoissonBracket(A, B), C)
order 1
One is undefined function
['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A, B), C)

In [482]:
test7 = PoissonBracket(test2,C)
test7

['x^{2}', 'p_{x} + x']
['\\lbrace x^{2},p_{x} + x\\rbrace', 'C']


PoissonBracket(PoissonBracket(x**2, p_x + x), C)

In [483]:
test7.doit()

PoissonBracket(PoissonBracket(x**2, p_x + x), C)
<class '__main__.PoissonBracket'>
(PoissonBracket(x**2, p_x + x), C)
PoissonBracket(PoissonBracket(x**2, p_x + x), C)
order 1
One is undefined function
['x^{2}', 'p_{x} + x']
['\\lbrace x^{2},p_{x} + x\\rbrace', 'C']


PoissonBracket(PoissonBracket(x**2, p_x + x), C)

In [484]:
test8 = PoissonBracket(test3,C)
test8

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A(x, p_x), B(y, p_y)), C)

In [485]:
test9 = PoissonBracket(test4,C)
test9

['x^{2}', 'p_{x} + x']
['\\lbrace x^{2},p_{x} + x\\rbrace', 'C']


PoissonBracket(PoissonBracket(x**2, p_x + x), C)

In [486]:
test9.doit()

PoissonBracket(PoissonBracket(x**2, p_x + x), C)
<class '__main__.PoissonBracket'>
(PoissonBracket(x**2, p_x + x), C)
PoissonBracket(PoissonBracket(x**2, p_x + x), C)
order 1
One is undefined function
['x^{2}', 'p_{x} + x']
['\\lbrace x^{2},p_{x} + x\\rbrace', 'C']


PoissonBracket(PoissonBracket(x**2, p_x + x), C)

In [487]:
test10 = PoissonBracket(test5,C)
test10

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C)

In [488]:
test10.free_symbols

{p_x, p_y, x, y}

In [489]:
test11 = PoissonBracket(test5,C(x,px))
test11

['A', 'B']
['C', '\\lbrace A,B\\rbrace']


-PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))

In [490]:
test11.args[1].order

-1

In [491]:
test11.doit()

PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))
<class '__main__.PoissonBracket'>
(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))
PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))
order 1
in Function
['A', 'B']
['C', '\\lbrace A,B\\rbrace']


-PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))

In [492]:
test12 = PoissonBracket(test5,C(x,px),coords=[x,y],mom=[px,py])
test12

['A', 'B']
['C', '\\lbrace A,B\\rbrace']


-PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))

In [493]:
PoissonBracket(B(x,px),A(x,px),coords=[x],mom=[px]).doit()

PoissonBracket(A(x, p_x), B(x, p_x))
<class '__main__.PoissonBracket'>
(A(x, p_x), B(x, p_x))
-Derivative(A(x, p_x), p_x)*Derivative(B(x, p_x), x) + Derivative(A(x, p_x), x)*Derivative(B(x, p_x), p_x)
order 1
in Function


-(-Derivative(A(x, p_x), p_x)*Derivative(B(x, p_x), x) + Derivative(A(x, p_x), x)*Derivative(B(x, p_x), p_x))

In [494]:
PoissonBracket(A,B)+ PoissonBracket(B,A)

0

In [495]:
test12.args, test12.args[1].order

((-1, PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))), -1)

In [496]:
test12.doit()

PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))
<class '__main__.PoissonBracket'>
(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))
PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))
order 1
in Function
['A', 'B']
['C', '\\lbrace A,B\\rbrace']


-PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))

In [448]:
test12.args[1].expr

['A', 'B']
['C', '\\lbrace A,B\\rbrace']


PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))

In [32]:
test12.doit()

['A', 'B']
['C', '\\lbrace A,B\\rbrace']


-PoissonBracket(C(x, p_x), PoissonBracket(A(x, p_x), B(x, y, p_y)))

In [1135]:
test13 = PoissonBracket(A(x,px)+B(x,y,py),C(x,px),coords=[x,y],mom=[px,py])
test13

['A + B', 'C']


PoissonBracket(A(x, p_x) + B(x, y, p_y), C(x, p_x))

In [1136]:
test13.doit(),

((Derivative(A(x, p_x), x) + Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) - Derivative(A(x, p_x), p_x)*Derivative(C(x, p_x), x),)

In [1137]:
test14 = PoissonBracket(A(x,px),B(x,y,py) + C(x,px),coords=[x,y],mom=[px,py])
test14

['A', 'B + C']


PoissonBracket(A(x, p_x), B(x, y, p_y) + C(x, p_x))

In [1138]:
test14.doit(),

(-Derivative(A(x, p_x), p_x)*(Derivative(B(x, y, p_y), x) + Derivative(C(x, p_x), x)) + Derivative(A(x, p_x), x)*Derivative(C(x, p_x), p_x),)

In [1139]:
test15 = PoissonBracket(A(x,px)*B(x,y,py),C(x,px),coords=[x,y],mom=[px,py])
test15

['A * B', 'C']


PoissonBracket(A(x, p_x)*B(x, y, p_y), C(x, p_x))

In [1140]:
test15.doit(),

((A(x, p_x)*Derivative(B(x, y, p_y), x) + Derivative(A(x, p_x), x)*B(x, y, p_y))*Derivative(C(x, p_x), p_x) - Derivative(A(x, p_x), p_x)*B(x, y, p_y)*Derivative(C(x, p_x), x),)

In [1141]:
test16 = PoissonBracket(A(x,px),B(x,y,py)*C(x,px),coords=[x,y],mom=[px,py])
test16

['A', 'B * C']


PoissonBracket(A(x, p_x), B(x, y, p_y)*C(x, p_x))

In [1142]:
test16.doit(),

(-Derivative(A(x, p_x), p_x)*(B(x, y, p_y)*Derivative(C(x, p_x), x) + Derivative(B(x, y, p_y), x)*C(x, p_x)) + Derivative(A(x, p_x), x)*B(x, y, p_y)*Derivative(C(x, p_x), p_x),)

In [1143]:
test17 = PoissonBracket(A(x,px)**3,B(x,y,py),coords=[x,y],mom=[px,py])
test17

['A^3', 'B']


PoissonBracket(A(x, p_x)**3, B(x, y, p_y))

In [1160]:
test17.doit()

-3*A(x, p_x)**3*Derivative(A(x, p_x), p_x)*A(x, p_x)**(-1)*Derivative(B(x, y, p_y), x)

In [1145]:
test18 = PoissonBracket(test12,D(x,y),coords=[x,y],mom=[px,py])
test18

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']
['\\lbrace \\lbrace A,B\\rbrace,C\\rbrace', 'D']


PoissonBracket(PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x)), D(x, y))

In [1152]:
test18.doit() == -((-Derivative(A(x, px), px)*Derivative(B(x, y, py), py, (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), py, x))*Derivative(C(x, px), px) + Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), py, x)*Derivative(C(x, px), x))*Derivative(D(x, y), y) - ((-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x))*Derivative(C(x, px), (px, 2)) + (-Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), (px, 2), x)*Derivative(B(x, y, py), x))*Derivative(C(x, px), px) + Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)*Derivative(C(x, px), px, x) + Derivative(A(x, px), (px, 3))*Derivative(B(x, y, py), x)*Derivative(C(x, px), x))*Derivative(D(x, y), x)

False

In [1153]:
test18.doit()

-((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), p_y, (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), p_y, x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), y) - ((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) + (-Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), (p_x, 2), x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), p_x, x) + Derivative(A(x, p_x), (p_x, 3))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), x)

In [1161]:
print_latex(test18.expr)

- \left(\left(- \frac{\partial}{\partial p_{x}} A{\left(x,p_{x} \right)} \frac{\partial^{3}}{\partial x^{2}\partial p_{y}} B{\left(x,y,p_{y} \right)} - \frac{\partial^{2}}{\partial x\partial p_{x}} A{\left(x,p_{x} \right)} \frac{\partial^{2}}{\partial x\partial p_{y}} B{\left(x,y,p_{y} \right)}\right) \frac{\partial}{\partial p_{x}} C{\left(x,p_{x} \right)} + \frac{\partial^{2}}{\partial p_{x}^{2}} A{\left(x,p_{x} \right)} \frac{\partial^{2}}{\partial x\partial p_{y}} B{\left(x,y,p_{y} \right)} \frac{\partial}{\partial x} C{\left(x,p_{x} \right)}\right) \frac{\partial}{\partial y} D{\left(x,y \right)} - \left(\left(- \frac{\partial}{\partial p_{x}} A{\left(x,p_{x} \right)} \frac{\partial^{2}}{\partial x^{2}} B{\left(x,y,p_{y} \right)} - \frac{\partial^{2}}{\partial x\partial p_{x}} A{\left(x,p_{x} \right)} \frac{\partial}{\partial x} B{\left(x,y,p_{y} \right)}\right) \frac{\partial^{2}}{\partial p_{x}^{2}} C{\left(x,p_{x} \right)} + \left(- \frac{\partial^{2}}{\partial p_{x}^{2}} A{\le

In [1156]:
test18.doit(),

(-((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), p_y, (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), p_y, x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), y) - ((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) + (-Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), (p_x, 2), x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), p_x, x) + Derivative(A(x, p_x), (p_x, 3))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), x),)

In [1147]:
test19 = PoissonBracket(test13,D(x,y),coords=[x,y],mom=[px,py])
test19

['A + B', 'C']
['\\lbrace A + B,C\\rbrace', 'D']


PoissonBracket(PoissonBracket(A(x, p_x) + B(x, y, p_y), C(x, p_x)), D(x, y))

In [1149]:
test19.doit(),

(-((Derivative(A(x, p_x), x) + Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) - Derivative(A(x, p_x), p_x)*Derivative(C(x, p_x), p_x, x) - Derivative(A(x, p_x), (p_x, 2))*Derivative(C(x, p_x), x) + Derivative(A(x, p_x), p_x, x)*Derivative(C(x, p_x), p_x))*Derivative(D(x, y), x) - Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), p_x)*Derivative(D(x, y), y),)

In [1085]:
test1.indep_coords,test2.indep_coords,test3.indep_coords, test4.indep_coords, test5.indep_coords

(None, None, None, [x], [x, y])

In [505]:
test6.indep_coords,test7.indep_coords,test8.indep_coords, test9.indep_coords, test10.indep_coords, test11.indep_coords

(None, None, None, None, None, None)

In [506]:
test1.indep_mom, test2.indep_mom, test3.indep_mom, test4.indep_mom, test5.indep_mom

(None, None, None, [p_x], [p_x, p_y])

In [507]:
test1.free_symbols, test2.free_symbols, test3.free_symbols, test4.free_symbols, test5.free_symbols

(set(), {p_x, x}, {p_x, p_y, x, y}, {p_x, x}, {p_x, p_y, x, y})

In [508]:
test6.free_symbols, test7.free_symbols, test8.free_symbols, test9.free_symbols, test10.free_symbols, test11.free_symbols

(set(),
 {p_x, x},
 {p_x, p_y, x, y},
 {p_x, x},
 {p_x, p_y, x, y},
 {p_x, p_y, x, y})

In [509]:
test1.expr, test2.expr, test3.expr, test4.expr, test5.expr

(PoissonBracket(A, B),
 PoissonBracket(x**2, p_x + x),
 PoissonBracket(A(x, p_x), B(y, p_y)),
 2*x,
 -Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x))

In [510]:
test6.expr, test7.expr, test8.expr, test9.expr, test10.expr, test11.expr, test12.expr

(PoissonBracket(PoissonBracket(A, B), C),
 PoissonBracket(PoissonBracket(x**2, p_x + x), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(y, p_y)), C),
 PoissonBracket(PoissonBracket(x**2, p_x + x), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x)),
 (-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))

In [511]:
test1.doit(), test2.doit(), test3.doit(), test4.doit(), test5.doit()

(PoissonBracket(A, B),
 PoissonBracket(x**2, p_x + x),
 PoissonBracket(A(x, p_x), B(y, p_y)),
 2*x,
 -Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x))

In [512]:
test6.doit(), test7.doit(), test8.doit(), test9.doit(), test10.doit(), test11.doit(), test12.doit()

(PoissonBracket(PoissonBracket(A, B), C),
 PoissonBracket(PoissonBracket(x**2, p_x + x), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(y, p_y)), C),
 PoissonBracket(PoissonBracket(x**2, p_x + x), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x)),
 (-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))

In [513]:
test4.doit()

2*x

In [514]:
test5.doit()

-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x)

In [515]:
test12.doit()

(-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x)

In [516]:
test13.doit()

(Derivative(A(x, p_x), x) + Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) - Derivative(A(x, p_x), p_x)*Derivative(C(x, p_x), x)

In [517]:
test13.expand(commutator=True)

<class '__main__.PoissonBracket'>
<class '__main__.PoissonBracket'>
[PoissonBracket(A(x, p_x), C(x, p_x)), PoissonBracket(B(x, y, p_y), C(x, p_x))]


PoissonBracket(A(x, p_x), C(x, p_x)) + PoissonBracket(B(x, y, p_y), C(x, p_x))

In [1162]:
test14

['A', 'B + C']


PoissonBracket(A(x, p_x), B(x, y, p_y) + C(x, p_x))

In [518]:
test14.expand(commutator=True)

PoissonBracket(A(x, p_x), B(x, y, p_y)) + PoissonBracket(A(x, p_x), C(x, p_x))

In [519]:
test15.expand(commutator=True)

A(x, p_x)*PoissonBracket(B(x, y, p_y), C(x, p_x)) + PoissonBracket(A(x, p_x), C(x, p_x))*B(x, y, p_y)

In [520]:
test16.expand(commutator=True).doit()

B(x, y, p_y)*PoissonBracket(A(x, p_x), C(x, p_x)) + PoissonBracket(A(x, p_x), B(x, y, p_y))*C(x, p_x)

In [1163]:
test17

['A^3', 'B']


PoissonBracket(A(x, p_x)**3, B(x, y, p_y))

In [1164]:
test17.expand(commutator=True).doit()

['A', 'B']
['A', 'B']
['A', 'B']


A(x, p_x)*PoissonBracket(A(x, p_x), B(x, y, p_y))*A(x, p_x) + A(x, p_x)**2*PoissonBracket(A(x, p_x), B(x, y, p_y)) + PoissonBracket(A(x, p_x), B(x, y, p_y))*A(x, p_x)**2

In [522]:
test18.doit()

-((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), p_y, (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), p_y, x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), y) - ((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) + (-Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), (p_x, 2), x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), p_x, x) + Derivative(A(x, p_x), (p_x, 3))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), x)

In [523]:
test18.expand(commutator=True)

PoissonBracket(PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x)), D(x, y))

In [524]:
test19.doit()

-((Derivative(A(x, p_x), x) + Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) - Derivative(A(x, p_x), p_x)*Derivative(C(x, p_x), p_x, x) - Derivative(A(x, p_x), (p_x, 2))*Derivative(C(x, p_x), x) + Derivative(A(x, p_x), p_x, x)*Derivative(C(x, p_x), p_x))*Derivative(D(x, y), x) - Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), p_x)*Derivative(D(x, y), y)

In [525]:
test19.expand(commutator=True)

<class '__main__.PoissonBracket'>
<class '__main__.PoissonBracket'>
[PoissonBracket(A(x, p_x), C(x, p_x)), PoissonBracket(B(x, y, p_y), C(x, p_x))]
<class '__main__.PoissonBracket'>
<class '__main__.PoissonBracket'>
[PoissonBracket(PoissonBracket(A(x, p_x), C(x, p_x)), D(x, y)), PoissonBracket(PoissonBracket(B(x, y, p_y), C(x, p_x)), D(x, y))]


PoissonBracket(PoissonBracket(A(x, p_x), C(x, p_x)), D(x, y)) + PoissonBracket(PoissonBracket(B(x, y, p_y), C(x, p_x)), D(x, y))

In [546]:
from sympy.printing.latex import print_latex
from sympy import latex, Derivative

In [531]:
test19.doit().__class__

sympy.core.add.Add

In [541]:
latex(test19)

'\\lbrace \\lbrace A{\\left(x,p_{x} \\right)} + B{\\left(x,y,p_{y} \\right)},C\\rbrace,D\\rbrace'

In [543]:
print_latex(test19)

\lbrace \lbrace A{\left(x,p_{x} \right)} + B{\left(x,y,p_{y} \right)},C\rbrace,D\rbrace


In [551]:
res12 = (-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x))*Derivative(C(x, px), px) + Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)*Derivative(C(x, px), x)

In [552]:
test12.doit() == res12

True